In [19]:
%pip install semantic-link-labs

StatementMeta(, 7af94af9-2535-4b99-be70-8d925d7cf588, 32, Finished, Available, Finished)


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [26]:
#Author Nalaka
# modified from https://bidiaries.com/simplify-shortcuts-bulk-creation-with-pyspark-semantic-link-labs-in-microsoft-fabric#heading-scenario-2-bulk-shortcut-creation-from-existing-source-tables

import sempy_labs
from datetime import datetime

# Parameters for source and destination
source_lakehouse = "maag_gold"
source_workspace = "AnalyticsPlatformSA"
destination_lakehouse = "sales_gold" # note: target lakehouse must exist
destination_workspace = "AnalyticsPlatformSpoke"

StatementMeta(, 7af94af9-2535-4b99-be70-8d925d7cf588, 40, Finished, Available, Finished)

In [47]:
# Fetch the list of tables from the source lakehouse
# This is done once and reused to improve perf, do not rerun cells below without refreshing this list
tables = sempy_labs.lakehouse.get_lakehouse_tables(
    lakehouse=source_lakehouse,
    workspace=source_workspace
)

# to filter just based on a specific schema
# tables = tables[tables['Schema Name'] == 'finance']

tables_in_destination = sempy_labs.lakehouse.get_lakehouse_tables(
    lakehouse=destination_lakehouse,
    workspace=destination_workspace
)

StatementMeta(, 7af94af9-2535-4b99-be70-8d925d7cf588, 64, Finished, Available, Finished)

In [48]:
# check the content to be worked on
# tables

StatementMeta(, 7af94af9-2535-4b99-be70-8d925d7cf588, 65, Finished, Available, Finished)

In [49]:
# Function to check for existing shortcuts
def shortcut_exists(destination_lakehouse, destination_workspace, shortcut_name, schema_name):
    exists = ((tables_in_destination['Schema Name'] == schema_name) & 
            (tables_in_destination['Table Name'] == shortcut_name)).any()
            
    return exists

StatementMeta(, 7af94af9-2535-4b99-be70-8d925d7cf588, 66, Finished, Available, Finished)

In [50]:
#what to do if the target shortcut already exists
# 'Replace' or 'Skip' 
shortcut_duplicate_behaviour = 'Skip'  # or 'Skip' - or any_other_string this gives the same behaviour as Abort as in the documentation from sem link labs

StatementMeta(, 7af94af9-2535-4b99-be70-8d925d7cf588, 67, Finished, Available, Finished)

In [51]:
# Iterate through each row in the DataFrame to process the 'Table Name'
for _, row in tables.iterrows():
    shortcut_name = row['Table Name']  # Extract 'Table Name' column as shortcut name
    schema_name = row['Schema Name']
    source_path = f"Tables/{schema_name}"
    destination_path = f"Tables/{schema_name}"
    
    # Skip processing for shortcut_name 'schema.json.gz'
    if shortcut_name == 'schema.json.gz':
        continue
    
    print(f"\n ∟ {datetime.now().strftime('%H:%M:%S')} Processing shortcut: {shortcut_name}")

    if shortcut_duplicate_behaviour == 'Replace':

        # Check if the shortcut already exists
        if shortcut_exists(destination_lakehouse, destination_workspace, shortcut_name, schema_name):
            try:
                # Delete the existing shortcut
                sempy_labs.lakehouse.delete_shortcut(
                        shortcut_name=shortcut_name,
                        lakehouse=destination_lakehouse,
                        workspace=destination_workspace,
                        shortcut_path=destination_path
                    )
                print(f"🟢 The shortcut {shortcut_name} in {schema_name} in {destination_workspace} has been deleted.")
            except Exception as e:
                print(f"Failed to delete existing shortcut '{shortcut_name}'. Error: {e}")
                continue  # Skip to the next shortcut if delete fails

        # Create a new shortcut
        try:
            sempy_labs.lakehouse.create_shortcut_onelake(
                table_name=shortcut_name,
                source_lakehouse=source_lakehouse,
                source_workspace=source_workspace,
                destination_lakehouse=destination_lakehouse,
                destination_workspace=destination_workspace, 
                source_path=source_path, 
                destination_path=destination_path
            )
            print(f"🟢 Shortcut created successfully for: '{shortcut_name}' with schema '{schema_name}'")
        except Exception as e:
            print(f"🔴 Failed to create shortcut for '{shortcut_name}'. Error: {e}")
    else:
        print(f"🟢 The shortcut named {shortcut_name} in schema {schema_name} in lakehouse {destination_workspace} has been skipped, already exists.")

print("\n\n 🟢🟢🟢 All shortcuts processed successfully.")

StatementMeta(, 7af94af9-2535-4b99-be70-8d925d7cf588, 68, Finished, Available, Finished)

∟ 17:44:16 Processing shortcut: account
🟢 The shortcut account in finance in AnalyticsPlatformSpoke has been skipped, already exists.

 ∟ 17:44:16 Processing shortcut: invoice
🟢 The shortcut invoice in finance in AnalyticsPlatformSpoke has been skipped, already exists.

 ∟ 17:44:16 Processing shortcut: payment
🟢 The shortcut payment in finance in AnalyticsPlatformSpoke has been skipped, already exists.

 ∟ 17:44:16 Processing shortcut: order
🟢 The shortcut order in salesadb in AnalyticsPlatformSpoke has been skipped, already exists.

 ∟ 17:44:16 Processing shortcut: orderline
🟢 The shortcut orderline in salesadb in AnalyticsPlatformSpoke has been skipped, already exists.

 ∟ 17:44:16 Processing shortcut: orderpayment
🟢 The shortcut orderpayment in salesadb in AnalyticsPlatformSpoke has been skipped, already exists.

 ∟ 17:44:16 Processing shortcut: order
🟢 The shortcut order in salesfabric in AnalyticsPlatformSpoke has been skipped, already exists.

 ∟ 17:44:16 Processing shortcut: ord